In [1]:
import navis
from navis.nbl.smat import Lookup2d, LookupDistDotBuilder, Digitizer

In [2]:
import numpy as np

# use a replicable random number generator
RNG = np.random.default_rng(2021)


def augment_neuron(
    nrn: navis.TreeNeuron, scale_sigma=0.1, translation_sigma=50, jitter_sigma=10
):
    nrn = nrn.copy(deepcopy=True)
    nrn.name += "_aug"
    dims = list("xyz")
    coords = nrn.nodes[dims].to_numpy()

    # translate whole neuron
    coords += RNG.normal(scale=translation_sigma, size=coords.shape[-1])
    # jitter individual coordinates
    coords += RNG.normal(scale=jitter_sigma, size=coords.shape)
    # rescale
    mean = np.mean(coords, axis=0)
    coords -= mean
    coords *= RNG.normal(loc=1.0, scale=scale_sigma)
    coords += mean

    nrn.nodes[dims] = coords
    return nrn


original = list(navis.example_neurons())
jittered = [augment_neuron(n) for n in original]

dotprops = [navis.make_dotprops(n, k=5, resample=False) for n in original + jittered]
matching_pairs = [[idx, idx + len(original)] for idx in range(len(original))]

In [3]:
builder = LookupDistDotBuilder(
    dotprops, matching_pairs, use_alpha=True, seed=2021
).with_bin_counts([8, 5])
smat = builder.build()
as_table = smat.to_dataframe()
as_table

,"[0.0,0.14588644700379452)","[0.14588644700379452,0.29912252023057806)","[0.29912252023057806,0.4802508950780032)","[0.4802508950780032,0.7351365037506351)","[0.7351365037506351,0.9988406500803395)"
"[2.0140850483155233,57.8493661406481)",0.486661,0.379042,0.387260,0.445648,0.855775
"[57.8493661406481,81.31283353333698)",0.404766,0.343702,0.406445,0.417802,0.799595
"[81.31283353333698,104.08576202392578)",0.257971,0.311827,0.217626,0.248743,1.013926
"[104.08576202392578,128.14104591262304)",0.255728,0.089663,0.171599,0.115997,1.296510
"[128.14104591262304,155.36119651794434)",-0.136171,-0.107249,-0.125751,0.252883,1.987175
"[155.36119651794434,202.6728515625)",-0.575078,-0.448307,-0.475147,-0.221016,1.407061
"[202.6728515625,395.9569088293945)",-1.025938,-0.948679,-0.863801,-0.620512,0.054148
"[395.9569088293945,4709.61474609375)",-0.615558,-0.737251,-0.679764,-0.454779,-0.197384


In [7]:
original_dps = dotprops[:len(original)]
new_dps = [
    navis.make_dotprops(
        augment_neuron(n),
        k=5,
        resample=False
    )
    for n in original
]
RNG.shuffle(new_dps)

result = navis.nblast(
    original_dps, new_dps,
    use_alpha=True, scores="mean", normalized=True,
    smat=smat,
    n_cores=1,
)
result.index = [dp.name for dp in original_dps]
result.columns = [dp.name for dp in new_dps]
result

NBLAST is optimized for data in microns and it looks like your queries are not in microns.
NBLAST is optimized for data in microns and it looks like your targets are not in microns.


Preparing:   0%|          | 0/1 [00:00<?, ?it/s]

NBlasting:   0%|          | 0/5 [00:00<?, ?it/s]

,754538881_aug,722817260_aug,1734350908_aug,1734350788_aug,754534424_aug
1734350788,-0.271504,-0.361381,-0.272018,0.159048,-0.325920
1734350908,-0.400432,-0.491376,0.858478,-0.437819,-0.216206
722817260,-0.157433,0.127931,-0.365195,-0.407436,-0.393940
754534424,-0.237769,-0.413532,-0.193794,-0.401148,0.857898
754538881,-0.021153,-0.213349,-0.200214,-0.303390,-0.115017
